In [1]:
import pandas as pd
import numpy as np

### [데이터타입 통합]
kopo_channel_seasonality_new.csv 파일을 불러온 후
selloutData 변수에 담으세요
이후 QTY컬럼→실수(float), 이외컬럼→문자(str)로 변경한다

In [2]:
# selloutData = pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")
selloutData = pd.read_csv("https://raw.githubusercontent.com/hyokwan/python-lecture/refs/heads/master/dataset/kopo_channel_seasonality_new.csv")
selloutData

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A60,PRODUCT4,201402,71.0
1,A60,PRODUCT59,201402,22275.0
2,A60,PRODUCT34,201402,4463.0
3,A60,PRODUCT47,201402,0.0
4,A60,PRODUCT56,201402,23.0
...,...,...,...,...
124653,A10,PRODUCT60,201630,824.0
124654,A10,PRODUCT56,201630,275.0
124655,A10,PRODUCT61,201630,0.0
124656,A10,PRODUCT12,201630,15021.0


In [3]:

selloutData.columns
selloutData.REGIONID = selloutData.REGIONID.astype(str)
selloutData.PRODUCT = selloutData.PRODUCT.astype(str)
selloutData.YEARWEEK = selloutData.YEARWEEK.astype(str)
selloutData.QTY = selloutData.QTY.astype(float)
# strColumnList = ["REGIONID","PRODUCT","YEARWEEK"]
# floatColumnList = ["QTY"]

# for i in range(0, len(strColumnList)):
#     eacColumn = strColumnList[i]
#     selloutData[ eacColumn]  = selloutData[ eacColumn ].astype(str)

### [불량 데이터 처리]
kopo_channel_seasonality_new.csv 자료를 담은
selloutData 변수에서
QTY컬럼 음수(반품)인 경우 0, 양수인 경우 기존 QTY 값
유지하는 로직을 적용하여 QTY_NEW 컬럼을 추가한다

In [4]:
selloutData['QTY_NEW'] = np.where(selloutData['QTY'] < 0, 0, selloutData['QTY'])

In [5]:
# selloutData.loc[ selloutData.QTY < 0 ]

### [데이터 통합]
selloutData 자료에서
YEAR, WEEK 컬럼을 생성하고 WEEK 컬럼 값이 52 이하인
데이터만 refinedSelloutData 변수에 저장한다


In [6]:
# 연도와 주차 컬럼을 생성한다.
selloutData['YEAR'] = selloutData['YEARWEEK'].astype(str).str[:4]
selloutData['WEEK'] = selloutData['YEARWEEK'].astype(str).str[4:]

In [7]:
refinedSelloutData = selloutData.loc [ selloutData.WEEK <= "52" ]

In [8]:
selloutData.shape
refinedSelloutData.shape

(123864, 7)

### 
refinedSelloutData 에서 →
지역, 상품, 연주차 컬럼순으로 오름차순 정렬하여
sortedData 변수에 담으세요


In [9]:
sortKey = ["REGIONID","PRODUCT","YEARWEEK"]
sortedData = refinedSelloutData.sort_values(by=sortKey,
                                            ascending=True,
                                            ignore_index=True)
sortedData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
0,A00,PRODUCT34,201401,661.0,661.0,2014,01
1,A00,PRODUCT34,201402,679.0,679.0,2014,02
2,A00,PRODUCT34,201403,578.0,578.0,2014,03
3,A00,PRODUCT34,201404,532.0,532.0,2014,04
4,A00,PRODUCT34,201405,516.0,516.0,2014,05
...,...,...,...,...,...,...,...
123859,A77,PRODUCT12,201648,4152.0,4152.0,2016,48
123860,A77,PRODUCT12,201649,5086.0,5086.0,2016,49
123861,A77,PRODUCT12,201650,5846.0,5846.0,2016,50
123862,A77,PRODUCT12,201651,4933.0,4933.0,2016,51


###  [지역, 상품, 연도 별 집계]
sortedData 에서 지역, 상품, 연도 단위
판매량(QTY_NEW) 의 평균 연산 후
groupData 변수에 담는다

이후 컬럼명을 QTY_MEAN로 변경한다


In [10]:
groupKey = ["REGIONID","PRODUCT","YEAR"]
groupData = sortedData.groupby(by=groupKey)["QTY_NEW"].agg(["mean"]).reset_index()
groupData.rename(columns={"mean":"QTY_MEAN"})
# groupData : 지역, 상품, 연도별 평균 거래량(1년 = 52주)
# refinedSelloutData : 지역, 상품, 연주차별 거래량

,REGIONID,PRODUCT,YEAR,QTY_MEAN
0,A00,PRODUCT34,2014,275.961538
1,A00,PRODUCT34,2015,86.634615
2,A00,PRODUCT34,2016,36.576923
3,A00,PRODUCT58,2014,2.673077
4,A00,PRODUCT58,2015,5.711538
...,...,...,...,...
2377,A77,PRODUCT1,2015,3030.019231
2378,A77,PRODUCT1,2016,3375.326923
2379,A77,PRODUCT12,2014,2035.788462
2380,A77,PRODUCT12,2015,3540.980769


In [11]:
refinedSelloutData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
0,A60,PRODUCT4,201402,71.0,71.0,2014,02
1,A60,PRODUCT59,201402,22275.0,22275.0,2014,02
2,A60,PRODUCT34,201402,4463.0,4463.0,2014,02
3,A60,PRODUCT47,201402,0.0,0.0,2014,02
4,A60,PRODUCT56,201402,23.0,23.0,2014,02
...,...,...,...,...,...,...,...
124653,A10,PRODUCT60,201630,824.0,824.0,2016,30
124654,A10,PRODUCT56,201630,275.0,275.0,2016,30
124655,A10,PRODUCT61,201630,0.0,0.0,2016,30
124656,A10,PRODUCT12,201630,15021.0,15021.0,2016,30


In [12]:
groupData.loc[ (groupData.REGIONID == "A00") &
               (groupData.PRODUCT == "PRODUCT34") &
               (groupData.YEAR == "2014") ]

,REGIONID,PRODUCT,YEAR,mean
0,A00,PRODUCT34,2014,275.961538


In [13]:
# refinedSelloutData와 groupData를
# [REGIONID, PRODUCT, YEAR] 키로 조인하여
# mergedData 변수에 아래와 같이 담는다

In [14]:
joinKey = ["REGIONID", "PRODUCT", "YEAR"]
mergedData = pd.merge(left = refinedSelloutData,
            right = groupData,
            on = joinKey,
            how = "inner")

# sortKey = ["REGIONID", "YEARWEEK", "PRODUCT"]
# mergedData = mergedData.sort_values(by=sortKey).reset_index(drop=True)
sortKey = ["REGIONID","YEARWEEK", "PRODUCT"]
mergedData = mergedData.sort_values(by=sortKey,
                       ascending=True,
                       ignore_index=True)
mergedData = mergedData.rename(columns = {"mean":"QTY_MEAN"})
mergedData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,QTY_MEAN
0,A00,PRODUCT34,201401,661.0,661.0,2014,01,275.961538
1,A00,PRODUCT58,201401,0.0,0.0,2014,01,2.673077
2,A00,PRODUCT59,201401,9988.0,9988.0,2014,01,27418.692308
3,A00,PRODUCT34,201402,679.0,679.0,2014,02,275.961538
4,A00,PRODUCT58,201402,0.0,0.0,2014,02,2.673077
...,...,...,...,...,...,...,...,...
123859,A77,PRODUCT12,201650,5846.0,5846.0,2016,50,4837.153846
123860,A77,PRODUCT1,201651,3671.0,3671.0,2016,51,3375.326923
123861,A77,PRODUCT12,201651,4933.0,4933.0,2016,51,4837.153846
123862,A77,PRODUCT1,201652,2759.0,2759.0,2016,52,3375.326923


In [15]:
mergedData["SEASONALITY"] = np.where( mergedData.QTY_MEAN != 0, mergedData.QTY / mergedData.QTY_MEAN, 0)
mergedData.loc[ (mergedData.REGIONID == "A60") &
                (mergedData.WEEK == "01") &
                (mergedData.PRODUCT == "PRODUCT4") ]

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,QTY_MEAN,SEASONALITY
112324,A60,PRODUCT4,201401,16.0,16.0,2014,01,29.134615,0.549175
112948,A60,PRODUCT4,201501,23.0,23.0,2015,01,71.711538,0.320729
113572,A60,PRODUCT4,201601,48.0,48.0,2016,01,32.615385,1.471698


In [ ]:
# mergedData를 활용하여
# SEASONALITY = QTY_NEW / QTY_MEAN
# SEASONALITY 컬럼을 생성한다.

# 마지막으로 [지역, 상품, 주차] 별
# SEASONALITY 평균 값을
# 산출하여 finalResult 변수에 담는다.

In [17]:
mergedData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,QTY_MEAN,SEASONALITY
0,A00,PRODUCT34,201401,661.0,661.0,2014,01,275.961538,2.395261
1,A00,PRODUCT58,201401,0.0,0.0,2014,01,2.673077,0.000000
2,A00,PRODUCT59,201401,9988.0,9988.0,2014,01,27418.692308,0.364277
3,A00,PRODUCT34,201402,679.0,679.0,2014,02,275.961538,2.460488
4,A00,PRODUCT58,201402,0.0,0.0,2014,02,2.673077,0.000000
...,...,...,...,...,...,...,...,...,...
123859,A77,PRODUCT12,201650,5846.0,5846.0,2016,50,4837.153846,1.208562
123860,A77,PRODUCT1,201651,3671.0,3671.0,2016,51,3375.326923,1.087598
123861,A77,PRODUCT12,201651,4933.0,4933.0,2016,51,4837.153846,1.019815
123862,A77,PRODUCT1,201652,2759.0,2759.0,2016,52,3375.326923,0.817402


In [18]:
mergedData["SEASONALITY"] = np.where( mergedData.QTY_MEAN != 0, mergedData.QTY / mergedData.QTY_MEAN, 0)

groupKey = ["REGIONID", "PRODUCT", "WEEK"]
finalResult = mergedData.groupby(by=groupKey)["SEASONALITY"].agg(["mean"]).reset_index()
finalResult = finalResult.rename(columns = {"mean" : "SEASONALITY_MEAN"})

finalResult

,REGIONID,PRODUCT,WEEK,SEASONALITY_MEAN
0,A00,PRODUCT34,01,1.570782
1,A00,PRODUCT34,02,1.755540
2,A00,PRODUCT34,03,1.319460
3,A00,PRODUCT34,04,1.490298
4,A00,PRODUCT34,05,1.061909
...,...,...,...,...
41283,A77,PRODUCT12,48,1.352712
41284,A77,PRODUCT12,49,1.094083
41285,A77,PRODUCT12,50,1.386116
41286,A77,PRODUCT12,51,1.255192


### 그룹바이

In [19]:
# 아무 생각 없이 통계 모델 중 이동 평균을 데이터에 적용했을 때
sortedData["MA5"] = \
    sortedData.QTY_NEW.rolling( window = 5, center=True, min_periods = 3 ).mean()
sortedData["SEASONALITY"] = sortedData.QTY_NEW / sortedData.MA5
groupKey = ["REGIONID", "PRODUCT", "WEEK"]
sortedData.groupby( by = groupKey )["SEASONALITY"].agg(["mean"]).reset_index()

,REGIONID,PRODUCT,WEEK,mean
0,A00,PRODUCT34,01,0.767107
1,A00,PRODUCT34,02,0.872595
2,A00,PRODUCT34,03,0.929402
3,A00,PRODUCT34,04,1.161136
4,A00,PRODUCT34,05,0.868462
...,...,...,...,...
41283,A77,PRODUCT12,48,1.087273
41284,A77,PRODUCT12,49,0.908397
41285,A77,PRODUCT12,50,1.063923
41286,A77,PRODUCT12,51,0.875609


In [20]:
sortKey = ["REGIONID","PRODUCT","YEARWEEK"]
sortedData = refinedSelloutData.sort_values( by = sortKey, ignore_index=True)

In [21]:
def rollingMa5(eachGroup):

# groupby를 생각하며 적용했을 때
# groupKey = ["REGIONID", "PRODUCT"]

# 디버깅
# groupData = sortedData.groupby(groupKey)
# groupData.groups
# list( groupData.groups[0] )
# eachGroup = groupData.get_group( list(groupData.groups)[0] )
# eachGroup = sortedData.loc[ (sortedData.REGIONID == "A00") &
#                             (sortedData.PRODUCT == "PRODUCT34") ]
    eachGroup.QTY_NEW.rolling( window = 5, center=True, min_periods = 3 ).mean()
    return eachGroup

In [22]:
groupData = sortedData.groupby( groupKey ).apply(rollingMa5)
groupData.reset_index(drop=True)
# groupData

<ipython-input-22-531f6422d965>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  groupData = sortedData.groupby( groupKey ).apply(rollingMa5)


,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
0,A00,PRODUCT34,201401,661.0,661.0,2014,01
1,A00,PRODUCT34,201501,151.0,151.0,2015,01
2,A00,PRODUCT34,201601,21.0,21.0,2016,01
3,A00,PRODUCT34,201402,679.0,679.0,2014,02
4,A00,PRODUCT34,201502,191.0,191.0,2015,02
...,...,...,...,...,...,...,...
123859,A77,PRODUCT12,201551,3661.0,3661.0,2015,51
123860,A77,PRODUCT12,201651,4933.0,4933.0,2016,51
123861,A77,PRODUCT12,201452,2080.0,2080.0,2014,52
123862,A77,PRODUCT12,201552,5601.0,5601.0,2015,52


In [23]:
# refinedSelloutData 부터 시작해서 한번
# 그룹바이 (regionid,product) 에 대해서면 이동평균 5구간 구현 후
# 최종 [regionid, product, week] 로 그룹바이 SEASONALITY 계산
# final 까지 만드는거를 해본다

In [24]:
def rollingMA5(eachGroup) :
    # 주차 기준 정렬
    eachGroup = eachGroup.sort_values("WEEK").copy()

    # 이동평균
    eachGroup["MA5"] = eachGroup.QTY_NEW.rolling( window = 5, center=True, min_periods = 3 ).mean()

    # SEASONALITY 계산
    eachGroup["SEASONALITY"] = np.where(eachGroup.MA5 != 0, eachGroup.QTY_NEW / eachGroup.MA5, 0)

    return eachGroup

groupKey = ["REGIONID", "PRODUCT"]
finalData = refinedSelloutData.groupby( by = groupKey ).apply(rollingMA5).reset_index(drop=True)

# SEASONALITY 계산
groupKey = ["REGIONID", "PRODUCT", "WEEK"]
finalData = finalData.groupby( by = groupKey )["SEASONALITY"].agg(["mean"]).reset_index()

finalData.rename(columns = {"mean" : "SEASONALITY_MEAN"}, inplace = True)
finalData

<ipython-input-24-8fbd9e12ee9f>:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  finalData = refinedSelloutData.groupby( by = groupKey ).apply(rollingMA5).reset_index(drop=True)


,REGIONID,PRODUCT,WEEK,SEASONALITY_MEAN
0,A00,PRODUCT34,01,0.976095
1,A00,PRODUCT34,02,1.288076
2,A00,PRODUCT34,03,1.198085
3,A00,PRODUCT34,04,1.291320
4,A00,PRODUCT34,05,1.304468
...,...,...,...,...
41283,A77,PRODUCT12,48,1.029636
41284,A77,PRODUCT12,49,0.915166
41285,A77,PRODUCT12,50,1.022353
41286,A77,PRODUCT12,51,0.916469


In [26]:
refinedSelloutData
# 01. 데이터프레임 정렬하기 (REGIONID,PRODUCT,YEARWEEK) 기준
sortKey = ["REGIONID","PRODUCT","YEARWEEK"]
sortedData = refinedSelloutData.sort_values(by=sortKey, ignore_index=True)
sortedData

# 02. 그룹화할 데이터 ("REGIONID", "PRODUCT")
groupKey = ["REGIONID", "PRODUCT"]
groupData = sortedData.groupby(by=groupKey)
eachGroup = groupData.get_group( list(groupData.groups)[0])
eachGroup

# 03. 이동평균 구하기
eachGroup["MA5"] = eachGroup.QTY_NEW.rolling(window=5, center=True, min_periods=3).mean()
eachGroup

# 04. 함수화하기
def rollingMa5(eachGroup):
    # eachGroup = groupData.get_group( list(groupData.groups)[0])
    # 이동평균 구하기
    eachGroup["MA5"] = eachGroup.QTY_NEW.rolling(window=5, center=True, min_periods=3).mean()
    eachGroup["SEASONALITY"] = np.where(eachGroup.MA5 == 0, 0, eachGroup.QTY_NEW / eachGroup.MA5)
    return eachGroup

# 05. 함수 적용하기
groupData = sortedData.groupby(by=groupKey).apply(rollingMa5)

# 06. 그룹인덱스 해제
groupData.reset_index(drop=True, inplace=True)

# 07. 그룹키 변경 ("REGIONID", "PRODUCT", "WEEK")
groupKeys = ["REGIONID", "PRODUCT", "WEEK"]

# 08. finalResult
finalResult = groupData.groupby(by=groupKeys)["SEASONALITY"].agg(["mean"])
finalResult

<ipython-input-26-89c05864fc56>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eachGroup["MA5"] = eachGroup.QTY_NEW.rolling(window=5, center=True, min_periods=3).mean()
<ipython-input-26-89c05864fc56>:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  groupData = sortedData.groupby(by=groupKey).apply(rollingMa5)


mean
REGIONID PRODUCT   WEEK          
A00      PRODUCT34 01    0.767107
                   02    0.872595
                   03    0.929402
                   04    1.161136
                   05    0.868462
...                           ...
A77      PRODUCT12 48    1.087273
                   49    0.908397
                   50    1.063923
                   51    0.875609
                   52    0.968998

[41288 rows x 1 columns]